In [5]:
from dynrules.models import StochasticDynamics
import matplotlib.pyplot as plt
from dynrules.data import get_cdiff_processed_logdata
import numpy as np
import torch
from pathlib import Path

In [4]:
def get_data(t, xlog, tind_first, device):
    # TODO: tind_first may not need to be part of data or create elsewhere?
    num_time, num_subj, num_taxa = xlog.shape   

    times = torch.from_numpy(t).to(torch.float)
    xstd = torch.from_numpy(np.nanstd(xlog, axis=0)).to(torch.float)
    xlog = torch.from_numpy(xlog).to(torch.float)
    xmean = torch.nanmean(xlog, dim=0)

    tstd, tmean = torch.std_mean(times)
    xnorm = (xlog-xmean)/(xstd + 1e-6)
    xnorm[torch.isnan(xnorm)] = 0      # TODO: need to think more about norm-representaiton and latent encoding for masked times...
    tnorm = (times-tmean)/(tstd + 1e-6)

    xnormflat = xnorm.reshape((num_time,-1))
    norm_data = torch.cat([tnorm[:,None].T, xnormflat.T]).T
    norm_data = norm_data.reshape((-1))

    data = {'times': times.to(device),  'log_abundance': xlog.to(device), 'norm_data': norm_data.to(device), 'x_norm': xnorm.to(device), \
            'tind_first': tind_first}
    return data


## load C. diff dataset

In [3]:
t, xlog, mask_times, taxa = get_cdiff_processed_logdata(basepath="../CDIFF_DATA")

## setup model

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [7]:
basepath = Path("./")

In [8]:
outpath = basepath / "cdiff_model_fit"
outpath.mkdir(exist_ok=True, parents=True)

In [9]:
xabs = np.exp(xlog)
max_range = 1.1*np.nanmax(xabs, axis=(0,1)) # max over TxS; final shape O

## TODO:
- clean up pipeline and finish inference example
- clean up post-processing and rule visualization
- maybe add tutorial just for rule and posterior visualization